
# Colab — Fashion-MNIST MLP (Proyecto Especialidad ML)

**Antes de empezar:** en Colab ve a **Runtime → Change runtime type → Hardware accelerator = GPU** → *Save*.

Este notebook ejecuta tu proyecto modular en Colab con GPU. Incluye tres formas de cargar tu proyecto (GitHub, Drive o ZIP), configura `PYTHONPATH`, corre `scripts/main.py`, y muestra curvas/QC.


## 1) Instalar dependencias mínimas

In [ ]:

# Mantén versiones por defecto de Colab para máxima compatibilidad con GPU
!pip -q install -U tensorflow numpy matplotlib pillow


## 2) Comprobar TensorFlow y GPU

In [ ]:

import tensorflow as tf
print("TensorFlow:", tf.__version__)
print("Dispositivos:", tf.config.list_physical_devices())
print("GPU disponible?:", bool(tf.config.list_physical_devices('GPU')))



## 3) Estructura esperada del proyecto

El notebook espera la **estructura modular** que vienes utilizando, por ejemplo:

```text
/content/Modulo7Clase1MarcoParra/
├── scripts/
│   └── main.py
└── src/
    ├── utils/
    │   ├── preprocessing.py
    │   └── data_loader.py
    ├── models/
    │   └── mlp.py
    ├── evaluator/
    │   └── train_eval.py
    └── visualizer/
        └── plots.py
```



## 4) Importar el proyecto (elige **UNA** opción)

### Opción A — Clonar desde GitHub
Descomenta la celda y reemplaza la URL por la de tu repo.


In [ ]:

# %%bash
# git clone https://github.com/usuario/Modulo7Clase1MarcoParra.git /content/Modulo7Clase1MarcoParra
# echo "Clonado en /content/Modulo7Clase1MarcoParra"



### Opción B — Montar Google Drive
Si subiste tu proyecto a Drive, móntalo y apunta `PROJECT_ROOT` a su carpeta.


In [ ]:

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Cambia la ruta a donde tengas el proyecto en tu Drive
PROJECT_ROOT = '/content/drive/MyDrive/Modulo7Clase1MarcoParra'



### Opción C — Subir un `.zip`
Sube un ZIP con la estructura anterior; se descomprime en `/content/Modulo7Clase1MarcoParra`.


In [ ]:

from google.colab import files
import zipfile, os

print("Selecciona tu archivo .zip del proyecto…")
uploaded = files.upload()  # elige único .zip
zip_name = next(iter(uploaded.keys()))
zip_path = f"/content/{zip_name}"

TARGET_DIR = '/content/Modulo7Clase1MarcoParra'
os.makedirs(TARGET_DIR, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zf:
    zf.extractall(TARGET_DIR)

PROJECT_ROOT = TARGET_DIR
print("Proyecto extraído en:", PROJECT_ROOT)
!find "$PROJECT_ROOT" -maxdepth 3 -type f -print


## 5) Configurar `PYTHONPATH` y establecer directorio de trabajo

In [ ]:

import sys, os
assert 'PROJECT_ROOT' in globals(), "Define PROJECT_ROOT usando una de las opciones anteriores."
assert os.path.isdir(PROJECT_ROOT), f"No existe PROJECT_ROOT: {PROJECT_ROOT}"

if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

os.chdir(PROJECT_ROOT)
print("Working dir:", os.getcwd())
print("PYTHONPATH OK")
!ls -R | sed -n '1,200p'


## 6) Entrenar (usa tus módulos `src/*` y `scripts/main.py`)

In [ ]:

from importlib import reload

# Imports de tu proyecto
from src.utils.preprocessing import load_preprocess
from src.models.mlp import build_mlp_course
from src.evaluator.train_eval import compile_model, train_model, evaluate_on_test
from src.visualizer.plots import plot_history_inline, plot_history_from_json, save_history_qc_report

from scripts import main as train_main
reload(train_main)

# Ejecutar entrenamiento (Adam + CCE, 10 épocas, bs=32, val_split=0.2)
train_main.main(out_dir="outputs")


## 7) Visualizar resultados y QC

In [ ]:

import os, glob
from PIL import Image
from IPython.display import display, Markdown

OUT = os.path.join(os.getcwd(), "outputs")
print("Outputs en:", OUT)
for p in glob.glob(os.path.join(OUT, "*")):
    print(" -", os.path.basename(p))

# Mostrar última imagen de curvas
curves = sorted(glob.glob(os.path.join(OUT, "*_curves.png")))
if curves:
    display(Image.open(curves[-1]))

# Mostrar QC Markdown (si existe)
qc_files = sorted(glob.glob(os.path.join(OUT, "*_history_qc.md")))
if qc_files:
    with open(qc_files[-1], "r", encoding="utf-8") as f:
        display(Markdown(f.read()))


## 8) (Opcional) Guardar outputs en Drive

In [ ]:

import shutil, os

DEST = '/content/drive/MyDrive/outputs_FashionMNIST_MLP'
os.makedirs(DEST, exist_ok=True)

SRC = os.path.join(os.getcwd(), 'outputs')
if os.path.isdir(SRC):
    for fname in os.listdir(SRC):
        s = os.path.join(SRC, fname)
        d = os.path.join(DEST, fname)
        if os.path.isfile(s):
            shutil.copy2(s, d)
    print("Outputs copiados a:", DEST)
else:
    print("No se encontró la carpeta outputs.")



## Notas
- Colab no usa *conda* por defecto; replicamos dependencias con `pip`.
- Si necesitas igualar versiones exactas de tu local, puedes fijarlas (p. ej. `tensorflow==2.13.*`), aunque Colab suele traer TF moderno con soporte GPU.
- Si editas scripts en vivo, usa `from importlib import reload; reload(modulo)` antes de re-ejecutar celdas.
